# オッズ比と相対リスク
原因系とリスク系の値が質的変数であった場合、クロス集計表を用いて影響の方向性を考察できる。そこで、影響を与える変数を要因とし、影響を与えられる変数を反応と呼ぶ。

ここで、使用する計算は「オッズ比」と「リスク比(相対リスク)」である。

オッズ比は要因ごとに非反応数を分母にして分子に反応数を計算し、正(または負)の要因を負(または正)の要因で割った数値。数値が1に近いほど相関が無い事が分かる。

リスク比は要因ごとに反応数を反応数と非反応数の合計値で割り、正(または負)の要因を負(または正)の要因で割った数値。リスク比はオッズ比と異なる点として反応数が小さくても要因ごとに違いが出やすくなる。

## ライブラリのインポート

In [1]:
import statsmodels.formula.api as smf
import statsmodels.api as sm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## データの確認
データセットは中村好一著『基礎から学ぶ 楽しい疫学 第4版』のp105にある集計表より作成。

In [2]:
df = pd.read_csv("LungCanser.csv")
df.head()

,Lighter,Smoke,LungCanser
0,1,1,0
1,1,1,0
2,1,1,0
3,1,1,0
4,1,1,0


## クロス集計表を作成

In [3]:
cross = pd.crosstab(df["Smoke"], df["LungCanser"])
cross

LungCanser,0,1
Smoke,,
0,70,30
1,30,70


In [4]:
crsary = cross.values
crsary

array([[70, 30],
       [30, 70]], dtype=int64)

## オッズ比の算出

In [5]:
odds = [[crsary[0][1]/crsary[0][0]], [crsary[1][1]/crsary[1][0]]]
odds_ratio = [odds[1][0]/odds[0][0]]
df_odds = pd.DataFrame(odds)
df_odds_ratio = pd.DataFrame(odds_ratio)
df_odds = pd.concat([df_odds,df_odds_ratio],axis=1)
df_odds.columns = ["オッズ", "オッズ比"]
df_odds.index = ["0", "1"]
df_odds.index.name = "LungCanser"
df_odds

,オッズ,オッズ比
LungCanser,,
0,0.428571,5.444444
1,2.333333,NaN


## リスク比の算出

In [6]:
risk = [[crsary[0][1]/(crsary[0][0]+crsary[0][1])], [crsary[1][1]/(crsary[1][0]+crsary[1][1])]]
risk_ratio = [risk[1][0]/risk[0][0]]
df_risk = pd.DataFrame(risk)
df_risk_ratio = pd.DataFrame(risk_ratio)
df_risk = pd.concat([df_risk,df_risk_ratio],axis=1)
df_risk.columns = ["リスク", "リスク比"]
df_risk.index = ["0", "1"]
df_risk.index.name = "LungCanser"
df_risk

,リスク,リスク比
LungCanser,,
0,0.3,2.333333
1,0.7,NaN


## 多変量解析におけるオッズ比
クロス集計表では2変数2水準でオッズ比を計算していたが、多変量解析を行う場合ではロジスティック回帰を用いて自然対数に算出された回帰係数を累乗する。

In [7]:
mod_glm = smf.glm(formula = "LungCanser ~ Smoke",
                  data = df.drop("Lighter", axis=1),
                  family = sm.families.Binomial()).fit()

In [8]:
coef = np.exp(mod_glm.params["Smoke"])
print("odds_ratio:%.2f"%(coef))

odds_ratio:5.44


In [9]:
mod_glm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:             LungCanser   No. Observations:                  200
Model:                            GLM   Df Residuals:                      198
Model Family:                Binomial   Df Model:                            1
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -122.17
Date:                Tue, 03 Jan 2023   Deviance:                       244.35
Time:                        13:01:16   Pearson chi2:                     200.
No. Iterations:                     4   Pseudo R-squ. (CS):             0.1517
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.8473      0.218     -3.883      0.000      -1.275      -0.420
Smoke          1.6946      0.309      5.491      0.000       1.090       2.299
==============================================================================
"""

### 変数名にスペースが入っている場合
変数名にスペースが入っている場合エラーが起きるため以下のように実装する

In [10]:
X = sm.add_constant(df["Smoke"])
model = sm.Logit(df["LungCanser"], X).fit_regularized()
model.summary()

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.6108643020553913
            Iterations: 12
            Function evaluations: 12
            Gradient evaluations: 12


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:             LungCanser   No. Observations:                  200
Model:                          Logit   Df Residuals:                      198
Method:                           MLE   Df Model:                            1
Date:                Tue, 03 Jan 2023   Pseudo R-squ.:                  0.1187
Time:                        13:01:16   Log-Likelihood:                -122.17
converged:                       True   LL-Null:                       -138.63
Covariance Type:            nonrobust   LLR p-value:                 9.637e-09
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.8473      0.218     -3.883      0.000      -1.275      -0.420
Smoke          1.6946      0.309      5.491      0.000       1.090       2.299
==============================================================================
"""